# CNN image classification in Pytorch using CIFAR-10 database

In [5]:
import torch
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms

In [9]:
batch_size_train = 4
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
trainset = CIFAR10('.pytorch/CIGAR10', train=True, transform=transform_train, download=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size_train, shuffle=True, num_workers=2)

testset = CIFAR10('.pytorch/CIGAR10', train=False, transform=transform_test, download=True)
testloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size_train, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified
